# 🎙️ Démonstration Qwen3-TTS-12Hz-0.6B-CustomVoice

Ce notebook présente le modèle de synthèse vocale **Qwen3-TTS** de l'équipe Qwen d'Alibaba.

## Fonctionnalités principales :
- **10 langues** : Chinois, Anglais, Japonais, Coréen, Allemand, Français, Russe, Portugais, Espagnol, Italien
- **9 voix premium** : Différents genres, âges, langues et dialectes
- **Contrôle par instructions** (modèle 1.7B) : Contrôlez le ton, l'émotion et le style de parole en langage naturel
- **Support du streaming** : Génération à faible latence
- **Haute qualité** : Synthèse vocale de pointe avec une forte compréhension contextuelle

⚠️ **Important** : Ce notebook nécessite un GPU. Allez dans `Exécution > Modifier le type d'exécution > GPU (T4 ou supérieur)`

## 1️⃣ Installation

Installation du paquet `qwen-tts` et des dépendances. Cela peut prendre quelques minutes.

In [ ]:
# Installation du paquet qwen-tts
!pip install -U qwen-tts soundfile -q

# Installation de flash-attention pour une inférence plus rapide (optionnel mais recommandé)
# La compilation peut prendre 5-10 minutes
!pip install flash-attn --no-build-isolation -q

In [ ]:
# Vérification de la disponibilité du GPU
import torch
print(f"Version PyTorch : {torch.__version__}")
print(f"CUDA disponible : {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU : {torch.cuda.get_device_name(0)}")
    print(f"VRAM : {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} Go")

## 2️⃣ Chargement du modèle

Nous chargeons le modèle **0.6B CustomVoice** qui supporte 9 voix préréglées premium.

Variantes de modèles disponibles :
- `Qwen/Qwen3-TTS-12Hz-0.6B-CustomVoice` - Plus petit, plus rapide (cette démo)
- `Qwen/Qwen3-TTS-12Hz-1.7B-CustomVoice` - Plus grand, supporte le contrôle par instructions
- `Qwen/Qwen3-TTS-12Hz-0.6B-Base` - Clonage de voix
- `Qwen/Qwen3-TTS-12Hz-1.7B-Base` - Clonage de voix (plus grand)
- `Qwen/Qwen3-TTS-12Hz-1.7B-VoiceDesign` - Conception de voix en langage naturel

In [ ]:
import torch
import soundfile as sf
from qwen_tts import Qwen3TTSModel
from IPython.display import Audio, display, Markdown
import os

# Création du répertoire de sortie
os.makedirs("audio_outputs", exist_ok=True)

# Chargement du modèle
print("Chargement du modèle Qwen3-TTS... (peut prendre quelques minutes au premier lancement)")

# Vérification de la disponibilité de flash_attention_2
try:
    import flash_attn
    attn_impl = "flash_attention_2"
    print("✅ Utilisation de Flash Attention 2 pour une inférence plus rapide")
except ImportError:
    attn_impl = "eager"
    print("⚠️ Flash Attention non disponible, utilisation de l'attention standard")

model = Qwen3TTSModel.from_pretrained(
    "Qwen/Qwen3-TTS-12Hz-0.6B-CustomVoice",
    device_map="cuda:0",
    dtype=torch.bfloat16,
    attn_implementation=attn_impl,
)

print("\n✅ Modèle chargé avec succès !")

## 3️⃣ Explorer les voix et langues disponibles

Voyons quelles voix et langues sont disponibles.

In [ ]:
# Récupération des locuteurs et langues supportés
speakers = model.get_supported_speakers()
languages = model.get_supported_languages()

print("🎤 LOCUTEURS DISPONIBLES :")
print("=" * 60)
speaker_info = {
    "Vivian": ("Femme", "Chinois", "Voix féminine jeune, vive et légèrement incisive"),
    "Serena": ("Femme", "Chinois", "Voix féminine jeune, chaleureuse et douce"),
    "Uncle_Fu": ("Homme", "Chinois", "Voix masculine mature avec un timbre grave et velouté"),
    "Dylan": ("Homme", "Chinois (Pékin)", "Voix masculine jeune de Pékin, claire et naturelle"),
    "Eric": ("Homme", "Chinois (Sichuan)", "Voix masculine enjouée de Chengdu, légèrement rauque"),
    "Ryan": ("Homme", "Anglais", "Voix masculine dynamique avec un rythme soutenu"),
    "Aiden": ("Homme", "Anglais", "Voix masculine américaine ensoleillée avec des médiums clairs"),
    "Ono_Anna": ("Femme", "Japonais", "Voix féminine espiègle avec un timbre léger et agile"),
    "Sohee": ("Femme", "Coréen", "Voix féminine chaleureuse avec une riche émotion"),
}

for speaker in speakers:
    if speaker in speaker_info:
        genre, langue_native, desc = speaker_info[speaker]
        print(f"\n  📌 {speaker}")
        print(f"     Genre : {genre} | Langue native : {langue_native}")
        print(f"     Description : {desc}")

print("\n" + "=" * 60)
print(f"\n🌍 LANGUES SUPPORTÉES ({len(languages)}) :")
print(", ".join(languages))

## 4️⃣ Génération de synthèse vocale de base

Génération de la parole avec un seul locuteur.

In [ ]:
def generate_and_play(text, language, speaker, filename, instruct=None):
    """Fonction utilitaire pour générer l'audio et l'afficher"""
    print(f"🎙️ Génération : \"{text[:50]}{'...' if len(text) > 50 else ''}\"")
    print(f"   Locuteur : {speaker} | Langue : {language}")
    if instruct:
        print(f"   Instruction : {instruct}")

    wavs, sr = model.generate_custom_voice(
        text=text,
        language=language,
        speaker=speaker,
        instruct=instruct if instruct else "",
    )

    filepath = f"audio_outputs/{filename}.wav"
    sf.write(filepath, wavs[0], sr)
    print(f"   ✅ Sauvegardé dans {filepath}")

    display(Audio(wavs[0], rate=sr))
    return wavs, sr

In [ ]:
# TTS basique en français
generate_and_play(
    text="Bonjour ! Bienvenue dans la démonstration de synthèse vocale Qwen3. Ce modèle peut générer une parole naturelle et expressive en plusieurs langues.",
    language="French",
    speaker="Serena",
    filename="01_francais_basique_serena"
)

In [ ]:
# TTS basique en chinois
generate_and_play(
    text="你好！欢迎使用Qwen3语音合成模型。这个模型可以生成自然、富有表现力的语音。",
    language="Chinese",
    speaker="Vivian",
    filename="02_chinois_basique_vivian"
)

In [ ]:
# TTS en français avec voix chinoise
generate_and_play(
    text="Bonjour, je peux aussi parler français avec un accent différent.",
    language="French",
    speaker="Vivian",
    filename="03_francais_accent_vivian"
)

In [ ]:
# TTS en allemand avec voix chinoise
generate_and_play(
    text="Hallo, ich kann auch Deutsch sprechen.",
    language="German",
    speaker="Vivian",
    filename="04_allemand_basique_vivian"
)

## 5️⃣ Démonstration multi-locuteurs

Comparaison de différentes voix avec le même texte.

In [ ]:
# Comparaison des voix en français
test_text = "Un renard roux et vif bondit par-dessus le chien endormi. Quelle scène amusante dans la campagne française !"

print("🎭 COMPARAISON DES VOIX EN FRANÇAIS\n")

for speaker in ["Serena", "Uncle_Fu"]:
    print(f"\n--- {speaker} ---")
    generate_and_play(
        text=test_text,
        language="French",
        speaker=speaker,
        filename=f"05_comparaison_fr_{speaker.lower()}"
    )

In [ ]:
# Comparaison des voix féminines chinoises
chinese_text = "春眠不觉晓，处处闻啼鸟。夜来风雨声，花落知多少。"

print("🎭 COMPARAISON DES VOIX FÉMININES CHINOISES\n")

for speaker in ["Vivian", "Serena"]:
    print(f"\n--- {speaker} ---")
    generate_and_play(
        text=chinese_text,
        language="Chinese",
        speaker=speaker,
        filename=f"04_comparison_{speaker.lower()}"
    )

## 6️⃣ Démonstration multilingue

Le même locuteur peut parler plusieurs langues ! Démonstration.

In [ ]:
# Serena parlant différentes langues (démonstration multilingue)
multilingual_samples = [
    ("French", "Bonjour ! Comment allez-vous aujourd'hui ?"),
    ("English", "Good morning! How are you doing today?"),
    ("German", "Guten Morgen! Wie geht es Ihnen heute?"),
    ("Spanish", "¡Buenos días! ¿Cómo está usted hoy?"),
    ("Italian", "Buongiorno! Come sta oggi?"),
    ("Portuguese", "Bom dia! Como você está hoje?"),
]

print("🌍 SERENA PARLANT PLUSIEURS LANGUES\n")

for i, (lang, text) in enumerate(multilingual_samples):
    print(f"\n--- {lang} ---")
    generate_and_play(
        text=text,
        language=lang,
        speaker="Serena",
        filename=f"06_multilingue_serena_{lang.lower()}"
    )

In [ ]:
# Locuteurs natifs dans leurs langues
native_samples = [
    ("Japanese", "Ono_Anna", "こんにちは！今日はとても良い天気ですね。散歩に行きましょう！"),
    ("Korean", "Sohee", "안녕하세요! 오늘 날씨가 정말 좋네요. 산책하러 갈까요?"),
    ("Chinese", "Dylan", "北京欢迎您！这里有故宫、长城、还有美味的北京烤鸭。"),
    ("Chinese", "Eric", "巴适得很！四川的火锅安逸惨了，欢迎来成都耍！"),
]

print("🌸 LOCUTEURS NATIFS DANS LEURS LANGUES\n")

for i, (lang, speaker, text) in enumerate(native_samples):
    print(f"\n--- {speaker} ({lang}) ---")
    generate_and_play(
        text=text,
        language=lang,
        speaker=speaker,
        filename=f"07_natif_{speaker.lower()}"
    )

## 7️⃣ Inférence par lots

Génération de plusieurs clips audio en un seul appel pour plus d'efficacité.

In [ ]:
# Inférence par lots - plusieurs textes à la fois
batch_texts = [
    "Premièrement, nous rassemblons tous les ingrédients pour notre recette.",
    "Ensuite, nous préchauffons le four à cent quatre-vingts degrés.",
    "Puis, nous mélangeons les ingrédients secs dans un grand saladier.",
    "Enfin, nous faisons cuire pendant vingt-cinq minutes jusqu'à coloration dorée.",
]

print("📦 DÉMONSTRATION D'INFÉRENCE PAR LOTS\n")
print(f"Génération de {len(batch_texts)} clips audio en un seul appel...\n")

wavs, sr = model.generate_custom_voice(
    text=batch_texts,
    language=["French"] * len(batch_texts),
    speaker=["Serena"] * len(batch_texts),
)

for i, (text, wav) in enumerate(zip(batch_texts, wavs)):
    print(f"\n📝 Étape {i+1} : {text}")
    filepath = f"audio_outputs/08_lot_etape{i+1}.wav"
    sf.write(filepath, wav, sr)
    display(Audio(wav, rate=sr))

## 8️⃣ Génération de texte long

Génération de passages plus longs.

In [ ]:
# Texte plus long - extrait d'une courte histoire
long_text = """Il était une fois, dans un petit village de Provence niché entre des collines couvertes de lavande et une rivière scintillante,
une jeune inventrice nommée Maya. Elle passait ses journées à bricoler des engrenages et des ressorts,
rêvant de machines capables de voler. Un matin, elle découvrit quelque chose d'extraordinaire dans son atelier :
un mystérieux plan qui semblait briller d'une lumière intérieure."""

print("📖 GÉNÉRATION DE TEXTE LONG\n")
generate_and_play(
    text=long_text,
    language="French",
    speaker="Serena",
    filename="09_histoire_longue"
)

## 9️⃣ Gestion du contenu technique et des nombres

Le modèle gère le contenu technique, les nombres et les caractères spéciaux.

In [ ]:
# Contenu technique avec nombres et équations
technical_samples = [
    "La vitesse de la lumière est d'environ deux cent quatre-vingt-dix-neuf millions sept cent quatre-vingt-douze mille mètres par seconde.",
    "Selon la célèbre équation d'Einstein, E égale mc carré.",
    "La réunion est prévue le quinze mars deux mille vingt-cinq à quinze heures trente.",
    "Le cours de l'action a augmenté de douze virgule cinq pour cent pour atteindre cent quarante-sept euros.",
]

print("🔢 CONTENU TECHNIQUE ET NOMBRES\n")

for i, text in enumerate(technical_samples):
    generate_and_play(
        text=text,
        language="French",
        speaker="Serena",
        filename=f"10_technique_{i+1}"
    )
    print()

## 🔟 Détection automatique de la langue

Définissez `language="Auto"` pour laisser le modèle détecter automatiquement la langue.

In [ ]:
# Détection automatique de la langue
auto_samples = [
    "Bonjour, ceci est une phrase en français.",
    "这是一个中文句子。",
    "これは日本語の文章です。",
]

print("🔍 DÉTECTION AUTOMATIQUE DE LA LANGUE\n")

for i, text in enumerate(auto_samples):
    print(f"\n--- Détection automatique pour : '{text}' ---")
    wavs, sr = model.generate_custom_voice(
        text=text,
        language="Auto",  # Laisser le modèle détecter
        speaker="Vivian",
    )
    filepath = f"audio_outputs/11_detection_auto_{i+1}.wav"
    sf.write(filepath, wavs[0], sr)
    display(Audio(wavs[0], rate=sr))

## 🎯 Espace de test interactif

Essayez votre propre texte avec différents locuteurs !

In [ ]:
# @title 🎮 Espace de test TTS interactif { display-mode: "form" }

# @markdown ### Entrez votre texte et vos paramètres :
text_input = "Bienvenue dans le futur de la technologie de synthèse vocale !" # @param {type:"string"}
speaker = "Serena" # @param ["Vivian", "Serena", "Uncle_Fu", "Dylan", "Eric", "Ryan", "Aiden", "Ono_Anna", "Sohee"]
language = "French" # @param ["Auto", "Chinese", "English", "Japanese", "Korean", "German", "French", "Russian", "Portuguese", "Spanish", "Italian"]

print(f"🎙️ Génération de la parole...")
print(f"   Texte : {text_input}")
print(f"   Locuteur : {speaker}")
print(f"   Langue : {language}")

wavs, sr = model.generate_custom_voice(
    text=text_input,
    language=language,
    speaker=speaker,
)

filepath = "audio_outputs/test_interactif.wav"
sf.write(filepath, wavs[0], sr)
print(f"\n✅ Généré ! Lecture de l'audio...")
display(Audio(wavs[0], rate=sr))

## 📥 Téléchargement des fichiers audio générés

Téléchargez tous les fichiers audio générés sous forme d'archive zip.

In [ ]:
import shutil
from google.colab import files

# Création de l'archive zip de toutes les sorties
shutil.make_archive("qwen3_tts_outputs", 'zip', "audio_outputs")

print("📦 Tous les fichiers audio ont été compressés !")
print("\n📁 Fichiers inclus :")
for f in sorted(os.listdir("audio_outputs")):
    if f.endswith('.wav'):
        print(f"   - {f}")

# Téléchargement
files.download("qwen3_tts_outputs.zip")

## 📚 Ressources supplémentaires

### Variantes de modèles

| Modèle | Fonctionnalités | Idéal pour |
|--------|-----------------|------------|
| `Qwen3-TTS-12Hz-0.6B-CustomVoice` | 9 voix préréglées, rapide | TTS général |
| `Qwen3-TTS-12Hz-1.7B-CustomVoice` | + Contrôle par instructions | TTS expressif |
| `Qwen3-TTS-12Hz-0.6B-Base` | Clonage de voix | Cloner n'importe quelle voix |
| `Qwen3-TTS-12Hz-1.7B-Base` | Clonage de voix (meilleur) | Clonage haute qualité |
| `Qwen3-TTS-12Hz-1.7B-VoiceDesign` | Conception de voix par texte | Création de voix personnalisées |

### Liens
- 🤗 [Collection Hugging Face](https://huggingface.co/collections/Qwen/qwen3-tts)
- 📝 [Dépôt GitHub](https://github.com/QwenLM/Qwen3-TTS)
- 📖 [Blog technique](https://qwen.ai/blog?id=qwen3tts-0115)
- 🎮 [Démo en ligne](https://huggingface.co/spaces/Qwen/Qwen3-TTS)

---
## 🎉 C'est terminé !

Vous avez exploré les principales fonctionnalités de **Qwen3-TTS-12Hz-0.6B-CustomVoice** :

- ✅ Plusieurs voix préréglées de haute qualité
- ✅ Support de 10 langues
- ✅ Inférence par lots
- ✅ Détection automatique de la langue
- ✅ Gestion du contenu technique

Pour le **contrôle des émotions/styles par instructions**, essayez le modèle **1.7B-CustomVoice**.
Pour le **clonage de voix**, essayez les modèles **Base**.
Pour la **conception de voix personnalisées**, essayez le modèle **VoiceDesign**.